**Installing necessary packages**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -q accelerate
!pip install -q torch
!pip install -q simpletransformers
!pip install -q datasets
!pip install -q evaluate
import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295

**Importing necessary llibraries**

In [ ]:
import numpy as np
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
df = pd.read_csv('/content/drive/MyDrive/sentiment.csv')
df.head(3)

,Sentiment,content
0,0,don't waste your money broken plates
1,0,waste of money! the phone keep falling as ther...
2,0,not specified not happy


**Checking class distribution**

In [ ]:
df['Sentiment'].value_counts()

0    4939
1    4404
Name: Sentiment, dtype: int64

**Dataset is balanced**

**Checking for and removing duplicates**

In [ ]:
print (df.shape)
df = df.drop_duplicates()
print (df.shape)

(9343, 2)
(9343, 2)


**Checking for missing values**

In [ ]:
df.isnull().sum()

Sentiment    0
content      0
dtype: int64

**Renaming target variable**

In [ ]:
df.rename(columns = {'Sentiment':'sentiment'}, inplace = True)
df.head(1)

,sentiment,content
0,0,don't waste your money broken plates


**Making train, test and validation splits**

In [ ]:
y_to_stratify_df = df['sentiment']
train_old, test = train_test_split (df, test_size=0.2, random_state=0, stratify = y_to_stratify_df, shuffle = True)

y_to_stratify_train = train_old['sentiment']
train, val = train_test_split (train_old, test_size=0.2, random_state=0, stratify = y_to_stratify_train, shuffle = True)

**Converting the dataset to arrows format for faster processing**

In [ ]:
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
val = Dataset.from_pandas(val)

In [ ]:
dataset = DatasetDict()

dataset['train'] = train
dataset['test'] = test
dataset['val'] = val
dataset

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'content', '__index_level_0__'],
        num_rows: 5979
    })
    test: Dataset({
        features: ['sentiment', 'content', '__index_level_0__'],
        num_rows: 1869
    })
    val: Dataset({
        features: ['sentiment', 'content', '__index_level_0__'],
        num_rows: 1495
    })
})

**Removing index column**

In [ ]:
dataset = dataset.remove_columns ('__index_level_0__')
dataset

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'content'],
        num_rows: 5979
    })
    test: Dataset({
        features: ['sentiment', 'content'],
        num_rows: 1869
    })
    val: Dataset({
        features: ['sentiment', 'content'],
        num_rows: 1495
    })
})

In [ ]:
train = train.remove_columns ('__index_level_0__')
test = test.remove_columns ('__index_level_0__')
val = val.remove_columns ('__index_level_0__')

**Intitializing bert tokenizer**

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

**Creating the function for tokenization**

In [ ]:
def tokenize_function(example):
    return tokenizer(example["content"], truncation=True)

**Tokenizing the dataset**

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/5979 [00:00<?, ? examples/s]

Map:   0%|          | 0/1869 [00:00<?, ? examples/s]

Map:   0%|          | 0/1495 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'content', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5979
    })
    test: Dataset({
        features: ['sentiment', 'content', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1869
    })
    val: Dataset({
        features: ['sentiment', 'content', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1495
    })
})

**Removing unncessary columns from tokenized datasets and renaming the target variable**

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["content"])
tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

**Initializing data collator and bert model**

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Defining a custom fuction for calculating classification metrics with each epoch**

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=1)
  acc = accuracy_score(labels, preds)
  return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

**Setting/Defining training arguments and incorporating them in the trainer**

In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch",
                                  num_train_epochs=10, per_device_train_batch_size=16,
                                  per_device_eval_batch_size=64, warmup_steps=500,
                                  weight_decay=0.01, logging_dir='./logs',
                                  logging_steps=10)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Model training**

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.153200,0.239804,0.929766,0.929238,0.936131,0.929766
2,0.131500,0.175067,0.956522,0.956527,0.956539,0.956522
3,0.113600,0.171905,0.952508,0.952542,0.953197,0.952508
4,0.003300,0.212450,0.960535,0.960466,0.961342,0.960535
5,0.000500,0.227244,0.958528,0.958422,0.959998,0.958528
6,0.000500,0.241243,0.959866,0.959880,0.959979,0.959866
7,0.045400,0.259096,0.959866,0.959863,0.959863,0.959866
8,0.031900,0.269792,0.963880,0.963849,0.964101,0.963880
9,0.000100,0.262295,0.964548,0.964538,0.964570,0.964548
10,0.000100,0.267132,0.963880,0.963861,0.963967,0.963880


TrainOutput(global_step=3740, training_loss=0.06101904762041927, metrics={'train_runtime': 926.3661, 'train_samples_per_second': 64.543, 'train_steps_per_second': 4.037, 'total_flos': 1957761571277220.0, 'train_loss': 0.06101904762041927, 'epoch': 10.0})

**Making predictions on test dataset**

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])

**Storing the target variable of test set in a variable named 'y_test' and converting it to a numpy array**

In [ ]:
y_test = tokenized_datasets['test']['labels']
y_test = np.array (y_test)

**Storing predictions in a variable named 'y_pred'**

In [ ]:
y_pred = np.argmax(predictions.predictions, axis=-1)

**Calculating model's evaluation metrics on test set**

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score

test_accuracy = accuracy_score (y_pred, y_test)
print ("Test Accuracy:", test_accuracy)
print ("Test Correct predictions:", sum (y_test == y_pred))
print ("Test Incorrect predictions:", sum (y_test != y_pred))
print ("Test F1 Score:", f1_score(y_test, y_pred, ))
print ('Test Precision Score: %.3f' % precision_score(y_test, y_pred))
print ('Test Recall Score: %.3f' % recall_score(y_test, y_pred))
print ("Classification Report test:\n", classification_report (y_test, y_pred, digits = 3))

Test Accuracy: 0.9641519529159979
Test Correct predictions: 1802
Test Incorrect predictions: 67
Test F1 Score: 0.9616485403548942
Test Precision Score: 0.970
Test Recall Score: 0.953
Classification Report test:
               precision    recall  f1-score   support

           0      0.959     0.974     0.966       988
           1      0.970     0.953     0.962       881

    accuracy                          0.964      1869
   macro avg      0.965     0.964     0.964      1869
weighted avg      0.964     0.964     0.964      1869



**Confusion matrix of test set**

In [ ]:
cm_test = confusion_matrix (y_test, y_pred)
print ("Test confusion matrix:\n", cm_test)

Test confusion matrix:
 [[962  26]
 [ 41 840]]


In [ ]:
df.sample(1)

,sentiment,content
4680,0,not recommended at all very bad worst quality ...


**Taking user input and making predictions from trained model**

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model='/content/test-trainer/checkpoint-3500/', return_all_scores=False)

user_input = input("Enter the text you want to predict: ")

prediction = classifier(user_input)

# Extract the label
label = prediction[0]['label']

if label == "LABEL_0":
  print('The review is negative.')
elif label == "LABEL_1":
  print('The review is positive.')

Enter the text you want to predict: very poor it is very bad skating board wast money tair is bad
The review is negative.
